# Pandas
## Data 765 tutoring

[pandas](https://pandas.pydata.org/) is an in memory tabular data frame library written in Python, Cython, and C and builds upon [NumPy](https://numpy.org/). `pandas` loads and manipulates data _in memory_ rather than storing temporary files or loading and operating on chunks. `pandas`' objects are tables that are closer to relational databases (i.e. SQL) than free form and nested objects (i.e. nested JSON or NoSQL). Data are stored as columns rather than rows. Columns are more efficient than rows due to data locality (i.e. what we spoke about in the NumPy mini lecture).

Data frames are omnipresent data structures in data analysis. Data frames are more than just abstractions over efficient math. They implement features to ease reshaping data or calculating metrics. For example, exploding a `pandas` DataFrame, joining DataFrames, or aggregating are all made easier by `pandas`.

`pandas` is a general purpose data frame library that fits a wide range of use cases. Other libraries implement specialized data frames that may be optimized for different workloads.

* [Polars](https://www.pola.rs/) is a speedy data frame library written in Rust with Python bindings. Polars is built on [Apache Arrow](https://github.com/apache/arrow-rs), a development framework for in memory data.
* [Apache Spark](https://spark.apache.org/) is designed for distributed, scalable workloads. Spark isn't written in Python, but it has bindings for Python, Scala, Java, R, and others.
* [Dask](https://dask.org/)'s API is similar to `pandas`. `Dask` scales up to clusters but scales down to single machines as well. `Dask` is useful for analyzing big data that don't fit into memory. 
* [datatable](https://github.com/h2oai/datatable) is a data frame library that is similar to R's [data.table](https://rdatatable.gitlab.io/data.table/). `datatable` uses optimized algorithms written in C that are based on `data.table`'s routines.

The above list isn't exhaustive but you get the idea.

# Data ingestion

`pandas` API is pretty intuitive and easy to use. The API itself is extensive which means that exhaustively covering everything in a few weeks is impossible. Instead, like the class lectures, I'll explain how to navigate and use the API so that you're able to waltz around the docs without being utterly overwhelmed.

Speaking of which: [the docs](https://pandas.pydata.org/docs/)

Don't read the docs end to end. Documentation should be purused for specific topics. Libraries usually have quick start guides or longer usage guides that can help with idiomatic use.

I'll use Pokémon [data from Kaggle](https://www.kaggle.com/mariotormo/complete-pokemon-dataset-updated-090420) compiled by Mario Tormo as well as Pokémon data I pulled from PokéAPI.

In [1]:
import pandas as pd
import numpy as np

pokemon = pd.read_csv("../data/pokedex.csv")

`pandas` is typically imported as the alias `pd`. `NumPy` is sometimes imported as well because `pandas` is built on `NumPy` and  it's generally useful to have it around.

 You can load several dreadful formats such as Excel, SAS, or SPSS as well as sane formats like CSV, [Parquet](https://en.wikipedia.org/wiki/Apache_Parquet), or [HDF](https://en.wikipedia.org/wiki/Hierarchical_Data_Format).

Generally speaking, you can look up "pandas FILEFORMAT" in your search engine of choice to find the function you need quickly. The documentation also has a [page listing the functions](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html). `pandas` can write to supported formats as well. A general paradigm is to clean up and reshape data then write the table out to a format for reuse.

`pandas`' I/O is flexible. Local files, remote files, file pointers, et cetera can be used to ingest data. For example, you can load the data I scraped for my thesis right from GitHub without saving a local copy.

In [ ]:
gamers = pd.read_csv("https://github.com/joshuamegnauth54/GamerDistributionThesis2020/raw/master/data/gamers_reddit_medium_2020.csv")

Or you can execute a SQL query on a server and create a DataFrame from the result.

Don't run this cell because the connection will fail.

In [ ]:
import psycopg2

with psycopg2.connect(database="pokemon", 
                      user="joshua",
                      password="fakepasswordilikecats",
                      host="localhost",
                      port="20000") as conn:
    
    query = """
            WITH pokemon_high_offense(species,
                          id,
                          base_stats,
                          ptype,
                          sp_atk,
                          sp_def,
                          atk,
                          def,
                          speed,
                          hp)
            AS (
                SELECT species,
                       id,
                       base_stats,
                       sp_atk,
                       sp_def,
                       atk,
                       def,
                       speed,
                       hp
                FROM pokemon
                WHERE (sp_atk >= percentile_cont(.5) WITHIN GROUP (ORDER BY sp_atk))
                       OR 
                       (atk >= percentile_cont(.5) WITHIN GROUP (ORDER BY atk))
                )
            SELECT *
            FROM pokemon_high_offense;
            """
    poke_hi_off = pd.read_sql(query, conn)

Each of these I/O functions have many parameters to tweak the ingestion process. For example, [pd.read_csv()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) supports:

* `delimiter`: providing a delimiter in case the engine fails to detect one
* `header`: specifying a different row to use as the header (column names)
* `names`: set column names
* `index_col`: column to use an index
* `usecols`: keep specific columns
* Parameters for parsing dates.

And **much** more. You should definitely take a look at the function docs for loading data as you will invariably have to specify parameters in the future.

# Series and DataFrames
[Series](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html) and [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) are `pandas`' main data structures. `DataFrames` consist of `Series`. `Series` are one dimensional ndarrays with additional exposed features to ease data ops.

`DataFrame`, as the documentation mentions, is a container for one or more `Series`. `DataFrame`s are more than a collection of ndarrays or `Series`. Recall the simple table example from the [data structures](https://github.com/joshuamegnauth54/data765-intro-python-tutoring/blob/main/notebooks/02-collections_basics.ipynb) mini-lecture. `DataFrame`s ease working with a similiar structure. For example, how would you calculate row-wise or column-wise statistics? You'd have to traverse a nested `list` by row or row and column in other to calculate a mean. This gets messy fast. Now imagine group bys, handling nulls, converting types, pivots, et cetera.

`NumPy` provides efficient iterators for traversing different dimensions of an ndarray, but features like the above are largely more data analysis focussed and thus better off in `pandas`.

## Attributes

From here on out almost everything applies to both `Series` and `DataFrame`s with the stipulation that a method called on a `Series` would only apply to that one object.

`Series` and `DataFrame`s both have properties similar to `NumPy`'s `ndarray`.

In [2]:
print(f"ndim: {pokemon.ndim}")
print(f"shape: {pokemon.shape}")
print(f"size: {pokemon.size}")
print(f"dtypes: {pokemon.dtypes}")

ndim: 2
shape: (1028, 51)
size: 52428
dtypes: Unnamed: 0            int64
pokedex_number        int64
name                 object
german_name          object
japanese_name        object
generation            int64
status               object
species              object
type_number           int64
type_1               object
type_2               object
height_m            float64
weight_kg           float64
abilities_number      int64
ability_1            object
ability_2            object
ability_hidden       object
total_points        float64
hp                  float64
attack              float64
defense             float64
sp_attack           float64
sp_defense          float64
speed               float64
catch_rate          float64
base_friendship     float64
base_experience     float64
growth_rate          object
egg_type_number       int64
egg_type_1           object
egg_type_2           object
percentage_male     float64
egg_cycles          float64
against_normal      float64
ag

These properties are less useful than they are with `ndarrays`. `ndim` is `2` because `pokemon` is a two dimensional table. The `shape` attribute shows (rows, columns).

`DataFrame`s have attributes and functions that are less "raw" than the above. For example, [DataFrame.columns](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.columns.html) returns an array-like data structure that contains the names of all of the columns. Similarly, the `index` attribute returns the index. Both attributes are mutable, but you have to replace them wholesale rather than in pieces. In other words, you can only replace all of the column names at once.

In [3]:
pokemon.columns

Index(['Unnamed: 0', 'pokedex_number', 'name', 'german_name', 'japanese_name',
       'generation', 'status', 'species', 'type_number', 'type_1', 'type_2',
       'height_m', 'weight_kg', 'abilities_number', 'ability_1', 'ability_2',
       'ability_hidden', 'total_points', 'hp', 'attack', 'defense',
       'sp_attack', 'sp_defense', 'speed', 'catch_rate', 'base_friendship',
       'base_experience', 'growth_rate', 'egg_type_number', 'egg_type_1',
       'egg_type_2', 'percentage_male', 'egg_cycles', 'against_normal',
       'against_fire', 'against_water', 'against_electric', 'against_grass',
       'against_ice', 'against_fight', 'against_poison', 'against_ground',
       'against_flying', 'against_psychic', 'against_bug', 'against_rock',
       'against_ghost', 'against_dragon', 'against_dark', 'against_steel',
       'against_fairy'],
      dtype='object')

`head()` and `tail()` show the first five rows of your frame or `Series`. You can pass a number as an argument to view more rows as well.

In [4]:
pokemon.head()

,Unnamed: 0,pokedex_number,name,german_name,japanese_name,generation,status,species,type_number,type_1,...,against_ground,against_flying,against_psychic,against_bug,against_rock,against_ghost,against_dragon,against_dark,against_steel,against_fairy
0,0,1,Bulbasaur,Bisasam,フシギダネ (Fushigidane),1,Normal,Seed Pokémon,2,Grass,...,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5
1,1,2,Ivysaur,Bisaknosp,フシギソウ (Fushigisou),1,Normal,Seed Pokémon,2,Grass,...,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5
2,2,3,Venusaur,Bisaflor,フシギバナ (Fushigibana),1,Normal,Seed Pokémon,2,Grass,...,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5
3,3,3,Mega Venusaur,Bisaflor,フシギバナ (Fushigibana),1,Normal,Seed Pokémon,2,Grass,...,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5
4,4,4,Charmander,Glumanda,ヒトカゲ (Hitokage),1,Normal,Lizard Pokémon,1,Fire,...,2.0,1.0,1.0,0.5,2.0,1.0,1.0,1.0,0.5,0.5


`info()` returns operational information about the `DataFrame` such as the type information.

In [5]:
pokemon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1028 entries, 0 to 1027
Data columns (total 51 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        1028 non-null   int64  
 1   pokedex_number    1028 non-null   int64  
 2   name              1028 non-null   object 
 3   german_name       938 non-null    object 
 4   japanese_name     938 non-null    object 
 5   generation        1028 non-null   int64  
 6   status            1028 non-null   object 
 7   species           1028 non-null   object 
 8   type_number       1028 non-null   int64  
 9   type_1            1028 non-null   object 
 10  type_2            542 non-null    object 
 11  height_m          1028 non-null   float64
 12  weight_kg         1027 non-null   float64
 13  abilities_number  1028 non-null   int64  
 14  ability_1         1025 non-null   object 
 15  ability_2         513 non-null    object 
 16  ability_hidden    810 non-null    object 


`info()` is a great way to confirm the types and missingness of your data. You should confirm that the integers are integers and that categoricals are strings/[Categorical](https://pandas.pydata.org/docs/reference/api/pandas.Categorical.html)/`object`s. You should also parse dates in order to leverage the full power of `NumPy`'s `datetime64` type. Your analysis could break if your type information is wrong.

Recently, the `pandas` team implemented a new `string` which is more efficient than storing text as `object`s. A new nullable `Int64` type replaces the old integer dtype. Previously, integers with `nan`s would parse as `float64`. As the lecture notes mention, you don't have to worry too much about these types just yet.

[DataFrame.astype](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.astype.html) is used to convert types.

In [6]:
pokemon = pokemon.astype({"name": "string",
                          "german_name": "string",
                          "japanese_name": "string",
                          "status": "category",
                          "species": "string",
                          "type_1": "category",
                          "type_2": "category",
                          "ability_1": "category",
                          "ability_2": "category"
                         })

[Renaming](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html) or [dropping](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html) rows or columns is useful to pare down your data set.

Both of these have several parameters that I won't go over (like all of the functions mentioned thus far).

In [7]:
pokemon.drop(columns=["Unnamed: 0",
                      "german_name",
                      "japanese_name",
                      "abilities_number",
                      "type_number"],
            inplace=True)

You should drop columns as early as possible, but I wanted to print the German and Japanese names. Also I forgot.

# Indexation

Retrieving and wrangling the exact data you need is a fundamental task of the data pipeline.

Columns may be selected by basic array style operators, `loc` and `iloc`, or the dot operator. Which method you use depends on the task. `loc` and `iloc` are more powerful and support more complicated filtering operations. They also allow mutation. 

In [8]:
print(f"Median special attack: {pokemon.sp_attack.median()}")

sp_def_75 = pokemon.sp_defense.quantile(.75)
sp_def_75 = pokemon["sp_defense"].quantile(.25)

Median special attack: 65.0
